In [31]:
!git clone https://github.com/rdyzakya/StanceClassifier

Cloning into 'StanceClassifier'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 164 (delta 80), reused 108 (delta 27), pack-reused 0
Receiving objects: 100% (164/164), 2.94 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [33]:
!pip install --upgrade pip

Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2


In [39]:
!pip install transformers
!pip install dataset
!pip install nltk
!pip install evaluate
!pip install seqeval
!pip install pandas
!pip install numpy
!pip install nlp-id

Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
  Using cached http://pip-notebook.modelarts.com:8888/repository/pypi/packages/a9/af/572a88cfacbd77aaa12dfe66dbd5a9de658c758e0894adbb0920750b0b21/nlp_id-0.1.13.0.tar.gz (7.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached http://pip-notebook.modelarts.com:8888/repository/pypi/packages/8b/99/b1ec652f2d60a13871a3053f312f9d78

In [28]:
!pip install --upgrade requests

Looking in indexes: http://pip-notebook.modelarts.com:8888/repository/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
modelarts 1.4.13 requires Pillow<=9.1.1, but you have pillow 9.2.0 which is incompatible.
modelarts 1.4.13 requires psutil~=5.9.1, but you have psutil 5.8.0 which is incompatible.
modelarts 1.4.13 requires tqdm<=4.64.0, but you have tqdm 4.64.1 which is incompatible.
modelarts 1.4.13 requires typing-extensions<=4.0.1, but you have typing-extensions 4.3.0 which is incompatible.
ma-cau 1.1.3 requires click==8.0.3, but you have click 8.1.3 which is incompatible.
ma-cau 1.1.3 requires pandas==1.3.4

In [44]:
%%writefile StanceClassifier/person_and_event_extraction/train_args.json

{
    "evaluation_strategy" : "epoch",
    "save_strategy" : "epoch",
    "learning_rate" : 5e-5,
    "gradient_accumulation_steps" : 2,
    "per_device_train_batch_size" : 32,
    "per_device_eval_batch_size" : 32,
    "num_train_epochs" : 5,
    "weight_decay" : 0.01,
    "load_best_model_at_end" : true,
    "metric_for_best_model" : "eval_overall_f1",
    "adam_epsilon" : 1e-8,
    "eval_accumulation_steps" : 10
}

Overwriting StanceClassifier/person_and_event_extraction/train_args.json


In [ ]:
!cd StanceClassifier/person_and_event_extraction && CUDA_VISIBLE_DEVICES=0 python main.py \
    --model_name_or_path bert-person \
    --max_len 256 \
    --training_args train_args.json \
    --n_gpu 0 \
    --do_train \
    --do_eval \
    --do_predict \
    --pred_batch_size 16 \
    --data_dir data/combine \
    --train "train" \
    --dev "val" \
    --test "test" \
    --output_dir bert-person

In [93]:
!cd StanceClassifier/person_and_event_extraction && CUDA_VISIBLE_DEVICES=0 python main.py \
    --model_name_or_path bert-person \
    --max_len 256 \
    --training_args train_args.json \
    --n_gpu 0 \
    --do_predict \
    --pred_batch_size 16 \
    --data_dir data/combine \
    --train "train" \
    --dev "val" \
    --test "test" \
    --output_dir bert-person

Predicting: 100%|█████████████████████████████| 207/207 [00:20<00:00, 10.26it/s]


In [94]:
path = "StanceClassifier/person_and_event_extraction/bert-person/predictions.txt"

In [95]:
f = open(path,'r',encoding="utf-8")
pred = f.read().strip().split("\n\n")
f.close()
pred = [[l.split('\t') for l in line.split('\n')] for line in pred]

In [96]:
y_true = []
y_pred = []

for i_text in range(len(pred)):
    for i_token in range((len(pred[i_text]))):
        try:
            y_true.append(pred[i_text][i_token][1])
            y_pred.append(pred[i_text][i_token][2])
        except:
            print("lahhh")
            print(i_text,i_token)

In [97]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

array([[    4,     0,  1315],
       [    0,     0,   164],
       [   59,    16, 42682]])

In [98]:
pred

[[['Tragedi', 'O', 'O'],
  ['serupa', 'O', 'O'],
  ['juga', 'O', 'O'],
  ['terjadi', 'O', 'O'],
  ['di', 'O', 'O'],
  ['Indonesia', 'O', 'O'],
  [',', 'O', 'O'],
  ['di', 'O', 'O'],
  ['Stadion', 'O', 'O'],
  ['Kanjuruhan', 'O', 'O'],
  [',', 'O', 'O'],
  ['awal', 'O', 'O'],
  ['bulan', 'O', 'O'],
  ['Oktober', 'O', 'O'],
  ['lalu', 'O', 'O'],
  [',', 'O', 'O']],
 [['Negara', 'O', 'O'],
  ['bagian', 'O', 'O'],
  ['itu', 'O', 'O'],
  ['memiliki', 'O', 'O'],
  ['1.200', 'O', 'O'],
  ['spesies', 'O', 'O'],
  ['tanaman', 'O', 'O'],
  ['asli', 'O', 'O'],
  ['dengan', 'O', 'O'],
  ['90', 'O', 'O'],
  ['persen', 'O', 'O'],
  ['di', 'O', 'O'],
  ['antaranya', 'O', 'O'],
  ['tidak', 'O', 'O'],
  ['ada', 'O', 'O'],
  ['di', 'O', 'O']],
 [['Pelaku', 'O', 'O'],
  ['usaha', 'O', 'O'],
  ['jasa', 'O', 'O'],
  ['keuangan', 'O', 'O'],
  ['yang', 'O', 'O'],
  ['masih', 'O', 'O'],
  ['memasarkan', 'O', 'O'],
  ['produknya', 'O', 'O'],
  ['dengan', 'O', 'O'],
  ['cara', 'O', 'O'],
  ['meresahkan', 'O', '